In [ ]:
import numpy as np

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_sun = 1.989e30  # Mass of the sun (kg)

class Particle:
    """A class representing a single particle in the simulation."""

    def __init__(self, mass, position, velocity):
        self.mass = mass
        self.position = np.array(position)
        self.velocity = np.array(velocity)
        self.force = np.zeros(3)

    def update_force(self, force):
        self.force += force

    def reset_force(self):
        self.force = np.zeros(3)

    def update_position(self, dt):
        self.velocity += self.force / self.mass * dt
        self.position += self.velocity * dt


class NBodySimulation:
    """A class to handle the N-body simulation."""

    def __init__(self, N, boundary_size, time_step):
        self.N = N
        self.boundary_size = boundary_size
        self.dt = time_step

        # Initialize particles with random positions and velocities
        self.particles = [
            Particle(M_sun,
                     position=np.random.uniform(-boundary_size/2, boundary_size/2, 3),
                     velocity=np.random.randn(3) * 1e4)  # Random initial velocity
            for _ in range(N)
        ]

    def compute_gravitational_force(self):
        """Compute gravitational forces between particles."""
        for particle in self.particles:
            particle.reset_force()  # Reset forces at the start of each step

        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Compute the gravitational force between particle i and j
                displacement = self.particles[j].position - self.particles[i].position
                distance = np.linalg.norm(displacement)

                if distance > 0:  # Avoid division by zero
                    force_magnitude = G * self.particles[i].mass * self.particles[j].mass / distance**2
                    force_direction = displacement / distance
                    force = force_magnitude * force_direction

                    self.particles[i].update_force(force)
                    self.particles[j].update_force(-force)  # Newton's third law


    def apply_boundary_conditions(self):
        inside_boundary_particles = []
        for particle in self.particles:
            # Check if the particle is inside the boundary
            if np.all(np.abs(particle.position) < self.boundary_size / 2):
                inside_boundary_particles.append(particle)
            else:
                # Reverse
                particle.velocity *= -1

        self.particles = inside_boundary_particles
        self.N = len(self.particles)

    def update_particles(self):
        """Update positions and velocities of particles."""
        for particle in self.particles:
            particle.update_position(self.dt)

    def run_simulation(self, steps):
        """Run the simulation for a certain number of steps."""
        for step in range(steps):
            self.compute_gravitational_force()
            self.update_particles()
            self.apply_boundary_conditions()

            if self.N == 0:
                print(f"All particles have exited the boundary. Stopping simulation at step {step}.")
                break

            print(f"Step {step}, remaining particles: {self.N}")

# Initialize and run the simulation
N = 100  # Number of particles
boundary_size = 1e21  # Size of the simulation box (meters)
time_step = 1e13  # Time step (seconds)

# Create the simulation
simulation = NBodySimulation(N, boundary_size, time_step)

# Run the simulation for 1000 steps
simulation.run_simulation(steps=1000)


In [ ]:
import numpy as np

# Constants
G = 6.67430e-11  # Gravitational constant (m^3 kg^-1 s^-2)
M_sun = 1.989e30  # Mass of the sun (kg)
N = 2 #number of particles in the simulation

In [ ]:
m = M_sun* np.ones((N)) #mass of the particles
v = np.zeros((N, 3)) #velocity of each particle in xyz coordinates
c = np.zeros ((N, 3)) #location of each particle in xyz coordinates

##Two Body Problem, 2D
In this case, it is very improbabile that any of these two particles go out of the boundries, so I leave this for the next part, which is the N-body problem.
For the two particles that will rotate around each other

In [ ]:
#This is an example for the initial velocities of two particles that will rotate around each other (2D).

#v [2, 3]
v_abs = 2 #absolute value of the velocity
#first particle
v[0, 0] = v_abs#x
v[0, 1] = v_abs #y
v[0, 2] = 0 #z

#second particle
v[1, 0] = -v_abs#x
v[1, 1] = -v_abs #y
v[1, 2] = 0 #z